In [1]:
import faiss
import random
import torch
import numpy as np
import torch.nn.functional as F

from tqdm.auto import tqdm
from torch.utils.data import DataLoader, TensorDataset, SequentialSampler
from datasets import load_from_disk, Dataset
from transformers import (
    AutoTokenizer,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments, RobertaModel, RobertaPreTrainedModel, BertModel, BertPreTrainedModel
)


In [2]:
# Anwer
class DenseRetrieval_with_Faiss:
    def __init__(self,
        args,
        dataset,
        tokenizer,
        p_encoder,
        q_encoder,
        num_neg=5,
        hard_neg=1,
        is_trained=False,
    ):
        """
        학습과 추론에 사용될 여러 셋업을 마쳐봅시다.
        """

        self.args = args
        self.dataset = dataset
        self.num_neg = num_neg

        self.tokenizer = tokenizer
        self.p_encoder = p_encoder
        self.q_encoder = q_encoder
        
        self.wiki_dataset = load_from_disk("/home/ubuntu/workspace/data/wiki_preprocessed_droped")
        
        if is_trained:
            pass
        else:
            self.prepare_in_batch_negative(num_neg=num_neg, hard_neg=hard_neg)

    def prepare_in_batch_negative(self,
        dataset=None,
        num_neg=5,
        hard_neg=1,
        k=100,
        tokenizer=None
    ):
        if num_neg < hard_neg:
            raise 'num_neg는 hard_neg보다 커야합니다.'
        wiki_datasets = self.wiki_dataset
        wiki_datasets.load_elasticsearch_index("text", host="localhost", port="9200", es_index_name="wikipedia_contexts")
        if dataset is None:
            dataset = self.dataset

        if tokenizer is None:
            tokenizer = self.tokenizer

        # 1. In-Batch-Negative 만들기
        # CORPUS를 np.array로 변환해줍니다.
        p_with_neg = []

        for c in tqdm(dataset):
            p_with_neg.append(c['context'])
            query = c['question']
            p_neg = []
            _, retrieved_examples = wiki_datasets.get_nearest_examples("text", query, k=k)
            for index in range(k):
                if retrieved_examples['document_id'][index] == c['document_id']:
                    continue
                p_neg.append(retrieved_examples['text'][index])
            p_with_neg.extend(p_neg[:hard_neg])
            p_with_neg.extend(random.sample(p_neg[50:], num_neg - hard_neg))
            assert len(p_with_neg) % (num_neg + 1) == 0, '데이터가 잘못 추가되었습니다.'

        # 2. (Question, Passage) 데이터셋 만들어주기
        q_seqs = tokenizer(
            dataset["question"],
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        p_seqs = tokenizer(
            p_with_neg,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        max_len = p_seqs["input_ids"].size(-1)
        p_seqs["input_ids"] = p_seqs["input_ids"].view(-1, num_neg+1, max_len)
        p_seqs["attention_mask"] = p_seqs["attention_mask"].view(-1, num_neg+1, max_len)
        p_seqs["token_type_ids"] = p_seqs["token_type_ids"].view(-1, num_neg+1, max_len)

        train_dataset = TensorDataset(
            p_seqs["input_ids"], p_seqs["attention_mask"], p_seqs["token_type_ids"], 
            q_seqs["input_ids"], q_seqs["attention_mask"], q_seqs["token_type_ids"]
        )

        self.train_dataloader = DataLoader(
            train_dataset,
            shuffle=True,
            batch_size=self.args.per_device_train_batch_size
        )

        valid_seqs = tokenizer(
            dataset["context"],
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        passage_dataset = TensorDataset(
            valid_seqs["input_ids"],
            valid_seqs["attention_mask"],
            valid_seqs["token_type_ids"]
        )
        self.passage_dataloader = DataLoader(
            passage_dataset,
            batch_size=self.args.per_device_train_batch_size
        )

    def build_faiss(self, num_clusters=16):
  
        """
        Note:
            위에서 Faiss를 사용했던 기억을 떠올려보면,
            Indexer를 구성해서 .search() 메소드를 활용했습니다.
            여기서는 Indexer 구성을 해주도록 합시다.
        """
        eval_batch_size = 8

        self.search_corpus = list(set([example['text'] for example in self.wiki_dataset]))
        p_encoder = self.p_encoder
        
        # Construt dataloader
        valid_p_seqs = self.tokenizer(
            self.search_corpus,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        valid_dataset = TensorDataset(
            valid_p_seqs["input_ids"],
            valid_p_seqs["attention_mask"],
            valid_p_seqs["token_type_ids"]
        )
        valid_sampler = SequentialSampler(valid_dataset)
        valid_dataloader = DataLoader(
            valid_dataset,
            sampler=valid_sampler,
            batch_size=eval_batch_size
        )

        # Inference using the passage encoder to get dense embeddeings
        p_embs = []

        with torch.no_grad():

            epoch_iterator = tqdm(
                valid_dataloader,
                desc="Iteration",
                position=0,
                leave=True
            )
            p_encoder.eval()

            for _, batch in enumerate(epoch_iterator):
                batch = tuple(t.cuda() for t in batch)

                p_inputs = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    "token_type_ids": batch[2]
                }
                
                outputs = p_encoder(**p_inputs).to("cpu").numpy()
                p_embs.extend(outputs)
        p_embs = np.array(p_embs)
        emb_dim = p_embs.shape[-1]

        quantizer = faiss.IndexFlatL2(emb_dim)
        self.indexer = faiss.IndexIVFScalarQuantizer(
            quantizer,
            quantizer.d,
            num_clusters,
            faiss.METRIC_L2
        )
        self.indexer.train(p_embs)
        self.indexer.add(p_embs)

    def train(self, args=None):
        if args is None:
            args = self.args
        batch_size = args.per_device_train_batch_size

        # Optimizer
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {"params": [p for n, p in self.p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
            {"params": [p for n, p in self.q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
        ]
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            eps=args.adam_epsilon
        )
        t_total = len(self.train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=args.warmup_steps,
            num_training_steps=t_total
        )

        # Start training!
        global_step = 0

        self.p_encoder.zero_grad()
        self.q_encoder.zero_grad()
        torch.cuda.empty_cache()

        train_iterator = tqdm(range(int(args.num_train_epochs)), desc="Epoch")
        # for _ in range(int(args.num_train_epochs)):
        for _ in train_iterator:

            with tqdm(self.train_dataloader, unit="batch") as tepoch:
                for batch in tepoch:

                    self.p_encoder.train()
                    self.q_encoder.train()
            
                    targets = torch.zeros(batch_size).long() # positive example은 전부 첫 번째에 위치하므로
                    targets = targets.to(args.device)

                    p_inputs = {
                        "input_ids": batch[0].view(batch_size * (self.num_neg + 1), -1).to(args.device),
                        "attention_mask": batch[1].view(batch_size * (self.num_neg + 1), -1).to(args.device),
                        "token_type_ids": batch[2].view(batch_size * (self.num_neg + 1), -1).to(args.device)
                    }
            
                    q_inputs = {
                        "input_ids": batch[3].to(args.device),
                        "attention_mask": batch[4].to(args.device),
                        "token_type_ids": batch[5].to(args.device)
                    }

                    # (batch_size*(num_neg+1), emb_dim)
                    p_outputs = self.p_encoder(**p_inputs)
                    # (batch_size*, emb_dim)
                    q_outputs = self.q_encoder(**q_inputs)

                    # Calculate similarity score & loss
                    p_outputs_t = torch.transpose(p_outputs.view(batch_size, self.num_neg + 1, -1), 1 , 2)
                    q_outputs = q_outputs.view(batch_size, 1, -1)

                    sim_scores = torch.bmm(q_outputs, p_outputs_t).squeeze()  #(batch_size, num_neg + 1)
                    sim_scores = sim_scores.view(batch_size, -1)
                    sim_scores = F.log_softmax(sim_scores, dim=1)

                    loss = F.nll_loss(sim_scores, targets)
                    tepoch.set_postfix(loss=f"{str(loss.item())}")

                    loss.backward()
                    optimizer.step()
                    scheduler.step()

                    self.p_encoder.zero_grad()
                    self.q_encoder.zero_grad()

                    global_step += 1

                    torch.cuda.empty_cache()

                    del p_inputs, q_inputs

    def get_relevant_doc(self, query, k=1):
        """
        Arguments:
            query (torch.Tensor):
                Dense Representation으로 표현된 query를 받습니다.
                문자열이 아님에 주의합시다.
            k (int, default=1):
                상위 몇 개의 유사한 passage를 뽑을 것인지 결정합니다.

        Note:
            받은 query를 이 객체에 저장된 indexer를 활용해서
            유사한 문서를 찾아봅시다.
        """

        valid_q_seqs = self.tokenizer(query, padding="max_length", truncation=True, return_tensors="pt").to("cuda")

        with torch.no_grad():
            q_encoder.eval()
            q_emb = q_encoder(**valid_q_seqs).to("cpu").numpy()
        
        q_emb = q_emb.astype(np.float32)
        D, I = self.indexer.search(q_emb, k)
        distances, index = D.tolist()[0], I.tolist()[0]
        
        distance_list, doc_list = [], []
        for d, i in zip(distances, index):
            distance_list.append(d)
            doc_list.append(self.search_corpus[i])

        return distance_list, doc_list

In [3]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 
  
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

In [4]:
class RobertaEncoder(RobertaPreTrainedModel):
    def __init__(self, config):
        super(RobertaEncoder, self).__init__(config)

        self.roberta = RobertaModel(config)
        self.init_weights()
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

In [5]:
# 평가를 안하니 검증데이터와 훈련데이터를 합칩니다.
train_dataset = load_from_disk('/home/ubuntu/workspace/data/train_dataset')
train = train_dataset['train'].to_dict()
valid = train_dataset['validation'].to_dict()
for key in train.keys():
  train[key].extend(valid[key])
train_dataset = Dataset.from_dict(train)
train_dataset

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__', 'chunks'],
    num_rows: 50
})

In [6]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=4,
    weight_decay=0.01
)
model_checkpoint = "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
p_encoder = BertEncoder.from_pretrained(model_checkpoint).to(args.device)
q_encoder = BertEncoder.from_pretrained(model_checkpoint).to(args.device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.de

In [7]:
retriever = DenseRetrieval_with_Faiss(
    args=args,
    dataset=train_dataset,
    tokenizer=tokenizer,
    p_encoder=p_encoder,
    q_encoder=q_encoder
)

  0%|          | 0/50 [00:00<?, ?it/s]

/home/ubuntu/workspace/mrc_venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [8]:
retriever.train()

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?batch/s]

  0%|          | 0/50 [00:00<?, ?batch/s]

  0%|          | 0/50 [00:00<?, ?batch/s]

  0%|          | 0/50 [00:00<?, ?batch/s]

In [9]:
retriever.build_faiss()

Iteration:   0%|          | 0/6996 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# 테스트
query = "리그오브레전드의 선수로, 페이커 데뷔전에서 솔킬을 따인 선수는?"
results = retriever.get_relevant_doc(query=query, k=5)

In [ ]:
results[1]

['돈키호테 도플라밍고는 어렸을 때 아버지의 결정으로 아버지, 어머니, 동생 로시난테(코라손)와 함께 성지 마리조아를 떠나 살기 시작했다. 처음에는 이전처럼 부유한 생활을 했지만, 우연히 자신이 천룡인이라는 것을 알게 된 사람들에 의해 집을 잃고 쫓겨다니는 신세가 되었다. 천룡인에 대한 사람들의 증오심은 두려움 못지않게 컸기 때문에, 도플라밍고와 그의 가족은 몇 번이고 죽을 위기를 맞았다. 도플라밍고가 8살이었을 때 어머니가 병을 앓다가 숨을 거두었고, 이 일로 아버지에 대해 분노를 느낀 도플라밍고는 10살 때 아버지를 자기 손으로 죽였다. 그리고 성지 마리조아로 돌아가 다시 천룡인으로 살려고 했으나 그들의 가족은 다른 천룡인들로부터 \'배신자\'로 낙인찍혀 두 번 다시 천룡인으로 살 수가 없었다. 이 이후로 도플라밍고는 천룡인을 포함한 전 세계에 대한 복수심을 품고 해적 생활을 시작했다. 해적 활동을 하던 도플라밍고는 천룡인에게 바쳐지는 공물을 나르는 배를 습격해 천상금으로 천룡인을 협박해 세계정부는 그에게 칠무해 자리를 내준다. 칠무해라는 특권인 동시에 족쇄가 되는 이름을 줘야 도플라밍고를 감시할 수 있기 때문이다. 한때 천룡인의 직함을 갖고 있었던 돈키호테 일족인지라 해군본부에서는 해당 인물에 대한 지속적인 관리가 필요했다. 한편으로 칠무해가 되기 전에는 자신의 부하 중 하나인 베르고를 해군에 입대시켜 해군 본부 중장으로 만들어 놓은 다음 베르고로 하여금 해군 본부에서 스파이를 하도록 시켰다. 13년전 코라손이 트라팔가 로에게 해군에게 도움을 요청하라고 했지만 그 많은 해군 중에서 하필이면 베르고에게 도움을 요청하는 바람에 코라손의 계획이 실패한다. 10년 전 칠무해가 된 도플라밍고는 과거에 돈키호테 일족이 다스렸던 드레스로자를 찾아가 국왕인 리쿠 왕에게 나라를 파는 조건으로 100억 베리를 요구한다. 리쿠 왕은 나라를 구하기 위해 국민들에게 돈을 내어 줄 것을 요구하고, 마침내 100억 베리가 모아진다. 하지만 도플라밍고는 리쿠 왕과 드레스로자의 병사들